<a href="https://colab.research.google.com/github/thomasshin/NLP_Study/blob/main/HuggingFace_NLP_Course/Main_NLP_Tasks/Fine_Tuning_a_masked_language_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets transformers[torch] accelerate -U

#Picking a pretrained model for masked language modeling

To get started, let’s pick a suitable pretrained model for masked language modeling. As shown in the following screenshot, you can find a list of candidates by applying the “Fill-Mask” filter on the Hugging Face Hub:

Although the BERT and RoBERTa family of models are the most downloaded, we’ll use a model called DistilBERT that can be trained much faster with little to no loss in downstream performance. This model was trained using a special technique called knowledge distillation, where a large “teacher model” like BERT is used to guide the training of a “student model” that has far fewer parameters. An explanation of the details of knowledge distillation would take us too far afield in this section, but if you’re interested you can read all about it in Natural Language Processing with Transformers (colloquially known as the Transformers textbook).

Let’s go ahead and download DistilBERT using the AutoModelForMaskedLM class:

In [2]:
from transformers import AutoModelForMaskedLM

model_ckpt = "distilbert-base-cased"
model = AutoModelForMaskedLM.from_pretrained(model_ckpt)

We can see how many parameters this model has by calling the num_parameters() method:

In [3]:
distilbert_num_parameters = model.num_parameters() / 1000000
print(f"'>>> DistilBERT number of parameters: {round(distilbert_num_parameters)}M'")
print(f"'>>> BERT number of parameters: 110M'")

'>>> DistilBERT number of parameters: 66M'
'>>> BERT number of parameters: 110M'


With around 66 million parameters, DistilBERT is approximately two times smaller than the BERT base model, which roughly translates into a two-fold speedup in training — nice! Let’s now see what kinds of tokens this model predicts are the most likely completions of a small sample of text:

In [4]:
text = "This is a great [MASK]."

As humans, we can imagine many possibilities for the [MASK] token, such as “day”, “ride”, or “painting”. For pretrained models, the predictions depend on the corpus the model was trained on, since it learns to pick up the statistical patterns present in the data. Like BERT, DistilBERT was pretrained on the English Wikipedia and BookCorpus datasets, so we expect the predictions for [MASK] to reflect these domains. To predict the mask we need DistilBERT’s tokenizer to produce the inputs for the model, so let’s download that from the Hub as well:

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

With a tokenizer and a model, we can now pass our text example to the model, extract the logits, and print out the top 5 candidates:

In [6]:
import torch

inputs = tokenizer(text, return_tensors="pt")
token_logits = model(**inputs).logits
# Find the location of [MASK] and extract its logits
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]
# Pick the [MASK] candidates with the highest logits
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()
for token in top_5_tokens:
  print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

'>>> This is a great achievement.'
'>>> This is a great success.'
'>>> This is a great deal.'
'>>> This is a great feat.'
'>>> This is a great celebration.'


In [7]:
token_logits.shape

torch.Size([1, 8, 28996])

We can see from the outputs that the model’s predictions refer to everyday terms, which is perhaps not surprising given the foundation of English Wikipedia. Let’s see how we can change this domain to something a bit more niche — highly polarized movie reviews!

#The dataset

To showcase domain adaptation, we’ll use the famous Large Movie Review Dataset (or IMDb for short), which is a corpus of movie reviews that is often used to benchmark sentiment analysis models. By fine-tuning DistilBERT on this corpus, we expect the language model will adapt its vocabulary from the factual data of Wikipedia that it was pretrained on to the more subjective elements of movie reviews. We can get the data from the Hugging Face Hub with the load_dataset() function from 🤗 Datasets:

In [8]:
from datasets import load_dataset

imdb_dataset = load_dataset("imdb")
imdb_dataset

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

We can see that the train and test splits each consist of 25,000 reviews, while there is an unlabeled split called unsupervised that contains 50,000 reviews. Let’s take a look at a few samples to get an idea of what kind of text we’re dealing with. As we’ve done in previous chapters of the course, we’ll chain the Dataset.shuffle() and Dataset.select() functions to create a random sample:

In [9]:
sample = imdb_dataset["train"].shuffle(seed=42).select(range(3))

for row in sample:
    print(f"\n'>>> Review: {row['text']}'")
    print(f"'>>> Label: {row['label']}'")


'>>> Review: There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier's plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it's the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...'
'>>> Label: 1'

'>>> Review: This movie is a great. The plot is very true to the book which is a classic written by Mark Twain. The movie starts of with a scene where Hank sings a song with a bunch of kids called "when you stu

Yep, these are certainly movie reviews, and if you’re old enough you may even understand the comment in the last review about owning a VHS version 😜! Although we won’t need the labels for language modeling, we can already see that a 0 denotes a negative review, while a 1 corresponds to a positive one.

Now that we’ve had a quick look at the data, let’s dive into preparing it for masked language modeling. As we’ll see, there are some additional steps that one needs to take compared to the sequence classification tasks we saw in Chapter 3. Let’s go!

#Preprocessing the data

For both auto-regressive and masked language modeling, a common preprocessing step is to concatenate all the examples and then split the whole corpus into chunks of equal size. This is quite different from our usual approach, where we simply tokenize individual examples. Why concatenate everything together? The reason is that individual examples might get truncated if they’re too long, and that would result in losing information that might be useful for the language modeling task!

So to get started, we’ll first tokenize our corpus as usual, but without setting the truncation=True option in our tokenizer. We’ll also grab the word IDs if they are available ((which they will be if we’re using a fast tokenizer, as described in Chapter 6), as we will need them later on to do whole word masking. We’ll wrap this in a simple function, and while we’re at it we’ll remove the text and label columns since we don’t need them any longer:

In [10]:
def tokenize_function(examples):
  result = tokenizer(examples["text"])
  if tokenizer.is_fast:
    result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
  return result

tokenized_datasets = imdb_dataset.map(
    tokenize_function, batched=True, remove_columns=["text", "label"]
)
tokenized_datasets

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (521 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 50000
    })
})

Since DistilBERT is a BERT-like model, we can see that the encoded texts consist of the input_ids and attention_mask that we’ve seen in other chapters, as well as the word_ids we added.

Now that we’ve tokenized our movie reviews, the next step is to group them all together and split the result into chunks. But how big should these chunks be? This will ultimately be determined by the amount of GPU memory that you have available, but a good starting point is to see what the model’s maximum context size is. This can be inferred by inspecting the model_max_length attribute of the tokenizer:

In [11]:
tokenizer.model_max_length

512

This value is derived from the tokenizer_config.json file associated with a checkpoint; in this case we can see that the context size is 512 tokens, just like with BERT.

So, in order to run our experiments on GPUs like those found on Google Colab, we’ll pick something a bit smaller that can fit in memory:

In [12]:
chunk_size = 128

Now comes the fun part. To show how the concatenation works, let’s take a few reviews from our tokenized training set and print out the number of tokens per review:

In [13]:
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_datasets["train"][:3]
for idx, sample in enumerate(tokenized_samples["input_ids"]):
  print(f"'>>> Review {idx} length: {len(sample)}'")

'>>> Review 0 length: 391'
'>>> Review 1 length: 322'
'>>> Review 2 length: 140'


We can then concatenate all these examples with a simple dictionary comprehension, as follows:

In [14]:
concatenated_examples = {
    k : sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}

max_len = len(concatenated_examples["input_ids"]) #total length

In [15]:
concatenated_examples

{'input_ids': [101,
  146,
  12765,
  146,
  6586,
  140,
  19556,
  19368,
  13329,
  118,
  162,
  21678,
  2162,
  17056,
  1121,
  1139,
  1888,
  2984,
  1272,
  1104,
  1155,
  1103,
  6392,
  1115,
  4405,
  1122,
  1165,
  1122,
  1108,
  1148,
  1308,
  1107,
  2573,
  119,
  146,
  1145,
  1767,
  1115,
  1120,
  1148,
  1122,
  1108,
  7842,
  1118,
  158,
  119,
  156,
  119,
  10148,
  1191,
  1122,
  1518,
  1793,
  1106,
  3873,
  1142,
  1583,
  117,
  3335,
  1217,
  170,
  5442,
  1104,
  2441,
  1737,
  107,
  6241,
  107,
  146,
  1541,
  1125,
  1106,
  1267,
  1142,
  1111,
  1991,
  119,
  133,
  9304,
  120,
  135,
  133,
  9304,
  120,
  135,
  1109,
  4928,
  1110,
  8663,
  1213,
  170,
  1685,
  3619,
  3362,
  2377,
  1417,
  14960,
  1150,
  3349,
  1106,
  3858,
  1917,
  1131,
  1169,
  1164,
  1297,
  119,
  1130,
  2440,
  1131,
  3349,
  1106,
  2817,
  1123,
  2209,
  1116,
  1106,
  1543,
  1199,
  3271,
  1104,
  4148,
  1113,
  1184,
  1103,
  190

Great, the total length checks out — so now let’s split the concatenated reviews into chunks of the size given by block_size. To do so, we iterate over the features in concatenated_examples and use a list comprehension to create slices of each feature. The result is a dictionary of chunks for each feature:

In [16]:
chunks = {k : [t[i : i + chunk_size] for i in range(0, max_len, chunk_size)] for k, t in concatenated_examples.items()}

for chunk in chunks["input_ids"]:
  print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 85'


As you can see in this example, the last chunk will generally be smaller than the maximum chunk size. There are two main strategies for dealing with this:

Drop the last chunk if it’s smaller than chunk_size.
Pad the last chunk until its length equals chunk_size.
We’ll take the first approach here, so let’s wrap all of the above logic in a single function that we can apply to our tokenized datasets:

In [17]:
def group_texts(examples):
  #concatenate all texts
  concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
  #Compute length of concatenated texts
  total_length = len(concatenated_examples[list(examples.keys())[0]])
  #We drop the last chunk if it's smaller than chunk_size
  total_length = (total_length // chunk_size) * chunk_size
  #Split by chunks of max_len
  result = {
      k: [t[i:i+chunk_size] for i in range(0, total_length,chunk_size)]
      for k, t in concatenated_examples.items()
  }
  # Create a new labels column
  result["labels"] = result["input_ids"].copy()

  return result

Note that in the last step of group_texts() we create a new labels column which is a copy of the input_ids one. As we’ll see shortly, that’s because in masked language modeling the objective is to predict randomly masked tokens in the input batch, and by creating a labels column we provide the ground truth for our language model to learn from.

Let’s now apply group_texts() to our tokenized datasets using our trusty Dataset.map() function:

In [18]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

You can see that grouping and then chunking the texts has produced many more examples than our original 25,000 for the train and test splits. That’s because we now have examples involving contiguous tokens that span across multiple examples from the original corpus. You can see this explicitly by looking for the special [SEP] and [CLS] tokens in one of the chunks:

In [19]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

"##e thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men. < br / > < br / > What kills me about I AM CURIOUS - YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, even then it's not shot like some cheaply made porno. While my countrymen mind find it shocking, in reality"

In this example you can see two overlapping movie reviews, one about a high school movie and the other about homelessness. Let’s also check out what the labels look like for masked language modeling:

In [20]:
tokenizer.decode(lm_datasets["train"][1]["labels"])

"##e thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men. < br / > < br / > What kills me about I AM CURIOUS - YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, even then it's not shot like some cheaply made porno. While my countrymen mind find it shocking, in reality"

##Fine-tuning DistilBERT with the Trainer API

Fine-tuning a masked language model is almost identical to fine-tuning a sequence classification model, like we did in Chapter 3. The only difference is that we need a special data collator that can randomly mask some of the tokens in each batch of texts. Fortunately, 🤗 Transformers comes prepared with a dedicated DataCollatorForLanguageModeling for just this task. We just have to pass it the tokenizer and an mlm_probability argument that specifies what fraction of the tokens to mask. We’ll pick 15%, which is the amount used for BERT and a common choice in the literature:

In [21]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

To see how the random masking works, let’s feed a few examples to the data collator. Since it expects a list of dicts, where each dict represents a single chunk of contiguous text, we first iterate over the dataset before feeding the batch to the collator. We remove the "word_ids" key for this data collator as it does not expect it:

In [22]:
samples = [lm_datasets["train"][i] for i in range(2)]

for sample in samples:
  _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
  print(f"\n' >>> {tokenizer.decode(chunk)}'")

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



' >>> [CLS] I rented [MASK] AM CURIO [MASK] - [MASK] [MASK]LOW from my flexible store because of all the [MASK] trains surrounded it when it was first [MASK] in 1967. I also heard that at first it was seized by U. S. customs if Guthrie ever tried to enter this country, therefore being a fan of films considered " [MASK] " I really had to see this [MASK] myself [MASK] < br / [MASK] < br / > The plot is centered around a young Swedish drama student named Lena who [MASK] to learn everything she [MASK] about life. In particular she wants to beta her attentions to making some sort of [MASK] on what the [MASK] Swed'

' >>> ##e thought about certain political issues [MASK] as [MASK] Vietnam War and race issues [MASK] the United States. [MASK] between asking politicians and ordinary [MASK]izens of [MASK] about their opinions on politics, [MASK] has sex [MASK] [MASK] drama [MASK] [MASK] classmates, and married [MASK]. < br / > < br / > [MASK] kills [MASK] about I [MASK] [MASK]URIOUS - 302ELLOW 

Nice, it worked! We can see that the [MASK] token has been randomly inserted at various locations in our text. These will be the tokens which our model will have to predict during training — and the beauty of the data collator is that it will randomize the [MASK] insertion with every batch!

One side effect of random masking is that our evaluation metrics will not be deterministic when using the Trainer, since we use the same data collator for the training and test sets. We’ll see later, when we look at fine-tuning with 🤗 Accelerate, how we can use the flexibility of a custom evaluation loop to freeze the randomness.

When training models for masked language modeling, one technique that can be used is to mask whole words together, not just individual tokens. This approach is called whole word masking. If we want to use whole word masking, we will need to build a data collator ourselves. A data collator is just a function that takes a list of samples and converts them into a batch, so let’s do this now! We’ll use the word IDs computed earlier to make a map between word indices and the corresponding tokens, then randomly decide which words to mask and apply that mask on the inputs. Note that the labels are all -100 except for the ones corresponding to mask words.

In [23]:
import collections
import numpy as np

from transformers import default_data_collator

wwm_probability = 0.2

def whole_masking_data_collator(features):
  for feature in features:
    word_ids = feature.pop("word_ids")

    # Create a map between words and corresponding token indices
    mapping = collections.defaultdict(list)
    current_word_index = -1
    current_word = None
    for idx, word_id in enumerate(word_ids):
      if word_id is not None:
        if word_id != current_word:
          current_word = word_id
          current_word_index += 1
        mapping[current_word_index].append(idx)

    # Randomly mask words
    mask = np.random.binomial(1, wwm_probability, (len(mapping), ))
    input_ids = feature["input_ids"]
    labels = feature["labels"]
    new_labels = [-100] * len(labels)
    for word_id in np.where(mask)[0]:
      word_id = word_id.item()
      for idx in mapping[word_id]:
        new_labels[idx] = labels[idx]
        input_ids[idx] = tokenizer.mask_token_id
    feature["labels"] = new_labels

  return default_data_collator(features)

Next, we can try it on the same samples as before:

In [24]:
samples = [lm_datasets["train"][i] for i in range(2)]
batch = whole_masking_data_collator(samples)
for chunk in batch["input_ids"]:
  print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] I rented [MASK] AM CURIOUS - YELLOW [MASK] my video [MASK] because [MASK] all the controversy that surrounded it when it was first released in 1967 [MASK] I [MASK] [MASK] that at first [MASK] was seized by U. S. [MASK] [MASK] it ever tried to enter this country, [MASK] [MASK] a fan of films considered " controversial " I really [MASK] [MASK] [MASK] this for myself. < br / > < [MASK] [MASK] > The [MASK] is [MASK] [MASK] [MASK] young [MASK] drama student named [MASK] who wants to learn everything she can about life. In particular she wants to focus [MASK] attentions [MASK] making [MASK] sort of documentary on what the average Swed'

'>>> ##e thought about certain political [MASK] such [MASK] the Vietnam [MASK] [MASK] race issues in the [MASK] States [MASK] In [MASK] asking politicians [MASK] ordinary denizens [MASK] [MASK] about [MASK] opinions on politics, she [MASK] sex with her drama teacher [MASK] [MASK], and [MASK] men [MASK] < br / > < br / > What kills [MASK] [MASK] [M

Now that we have two data collators, the rest of the fine-tuning steps are standard. Training can take a while on Google Colab if you’re not lucky enough to score a mythical P100 GPU 😭, so we’ll first downsample the size of the training set to a few thousand examples. Don’t worry, we’ll still get a pretty decent language model! A quick way to downsample a dataset in 🤗 Datasets is via the Dataset.train_test_split() function that we saw in Chapter 5:

In [25]:
train_size = 20000
test_size = int(0.1 * train_size)

downsampled_dataset = lm_datasets["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)

downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 2000
    })
})

This has automatically created new train and test splits, with the training set size set to 10,000 examples and the validation set to 10% of that — feel free to increase this if you have a beefy GPU! The next thing we need to do is log in to the Hugging Face Hub. If you’re running this code in a notebook, you can do so with the following utility function:

In [26]:
from huggingface_hub import notebook_login

notebook_login()

In [30]:
from transformers import TrainingArguments

batch_size=64
# Show the training loss with every epoch
logging_steps = len(downsampled_dataset["train"]) // batch_size
model_name = model_ckpt.split("/")[-1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-finetuned-imdb",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=True,
    fp16=True,
    logging_steps=logging_steps
)

In [31]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=downsampled_dataset["train"],
    eval_dataset=downsampled_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [32]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 29.44


In [33]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.686400,2.451071
2,2.534800,2.412327
3,2.491800,2.378330


TrainOutput(global_step=939, training_loss=2.5708478789486953, metrics={'train_runtime': 118.8738, 'train_samples_per_second': 504.737, 'train_steps_per_second': 7.899, 'total_flos': 1988347023360000.0, 'train_loss': 2.5708478789486953, 'epoch': 3.0})

In [34]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 10.94


In [35]:
trainer.push_to_hub()

pytorch_model.bin:   0%|          | 0.00/263M [00:00<?, ?B/s]

'https://huggingface.co/ShinDC/distilbert-base-cased-finetuned-imdb/tree/main/'